In [14]:
import pyautogui as pa
import keyboard
import mouse
import time

# Listas para armazenar as ações e timestamps
actions = []
timestamps = []

# Tempo inicial da gravação
start_time = time.time()

# Variável para rastrear se o botão esquerdo está pressionado
is_dragging = False

# Define o intervalo mínimo de tempo entre gravações de ações em segundos
min_interval = 0.05  # 50ms

# Variáveis para controlar o intervalo de gravação das ações
last_mouse_time = 0
last_keyboard_time = 0

def record_mouse_action(event):
    global is_dragging, last_mouse_time
    current_time = time.time()

    if current_time - last_mouse_time < min_interval:
        return  # Evita registrar ações muito frequentes

    if isinstance(event, mouse.ButtonEvent):
        x, y = pa.position()  # Pega a posição atual do mouse
        if event.event_type == "down" and event.button == "left":
            is_dragging = True  # Inicia o rastreamento de arrasto
            timestamps.append(current_time)
            actions.append(f"pa.mouseDown({x}, {y})  # Mouse down at ({x}, {y})")
        elif event.event_type == "up" and event.button == "left":
            is_dragging = False  # Finaliza o rastreamento de arrasto
            timestamps.append(current_time)
            actions.append(f"pa.mouseUp({x}, {y})  # Mouse up at ({x}, {y})")

    # Registra movimentos do mouse enquanto está arrastando
    elif isinstance(event, mouse.MoveEvent):
        if is_dragging:  # Somente registra movimentos durante o arrasto
            x, y = pa.position()
            timestamps.append(current_time)
            actions.append(f"pa.moveTo({x}, {y})  # Mouse move during drag")
    
    last_mouse_time = current_time

def record_keyboard_action(event):
    global last_keyboard_time
    current_time = time.time()

    if current_time - last_keyboard_time < min_interval:
        return  # Evita registrar ações muito frequentes

    if event.event_type == "down":
        combination = keyboard.get_hotkey_name()  # Captura combinações de teclas
        if combination == "esc":  # Finaliza a gravação ao pressionar "ESC"
            return True
        if "+" in combination:  # Combinação de teclas (ex.: Ctrl+C)
            keys = ", ".join([f'"{key}"' for key in combination.split("+")])
            timestamps.append(current_time)
            actions.append(f"pa.hotkey({keys})  # Hotkey: {combination}")
        elif len(event.name) == 1:  # Caracteres imprimíveis
            timestamps.append(current_time)
            actions.append(f"pa.write('{event.name}')  # Type: {event.name}")
        else:  # Teclas especiais ou únicas
            timestamps.append(current_time)
            actions.append(f"pa.press('{event.name}')  # Key press: {event.name}")
    
    last_keyboard_time = current_time
    return False

try:
    print("Iniciando gravação... Pressione ESC para parar.")

    # Inicia os listeners para mouse e teclado
    mouse.hook(record_mouse_action)
    keyboard.hook(record_keyboard_action)

    # Espera até que ESC seja pressionado
    keyboard.wait("esc")

finally:
    # Finaliza os listeners
    mouse.unhook_all()
    keyboard.unhook_all()

    # Gera o código PyAutoGUI
    print("\nAções gravadas:")
    previous_time = start_time
    for i, action in enumerate(actions):
        # Calcula o intervalo de tempo desde a última ação
        delay = timestamps[i] - previous_time
        print(f"time.sleep({delay:.2f})")  # Adiciona o delay ao terminal
        print(action)
        previous_time = timestamps[i]

    # Salva o macro em um arquivo
    with open("macroV2.py", "w") as file:
        file.write("import pyautogui as pa\n")
        file.write("import time\n\n")
        file.write("def play_macro():\n")
        previous_time = start_time
        for i, action in enumerate(actions):
            delay = timestamps[i] - previous_time
            file.write(f"    time.sleep({delay:.2f})\n")
            file.write(f"    {action}\n")
            previous_time = timestamps[i]
        file.write("\nif __name__ == '__main__':\n")
        file.write("    play_macro()\n")

    print("\nAs ações foram salvas no arquivo 'macro.py'.")

Iniciando gravação... Pressione ESC para parar.

Ações gravadas:
time.sleep(1.19)
pa.mouseDown(-1149, 839)  # Mouse down at (-1149, 839)
time.sleep(0.11)
pa.moveTo(-1153, 838)  # Mouse move during drag
time.sleep(0.42)
pa.mouseUp(-1159, 830)  # Mouse up at (-1159, 830)
time.sleep(1.09)
pa.press('ctrl')  # Key press: ctrl
time.sleep(0.21)
pa.hotkey("ctrl", "c")  # Hotkey: ctrl+c

As ações foram salvas no arquivo 'macro.py'.
